In [2]:
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'orderbot.settings')

'orderbot.settings'

In [3]:
from django.db import models
from django.contrib.auth.models import User


class Product(models.Model):
    product_name = models.CharField(max_length=255, null=False)
    quantity = models.IntegerField(null=False)
    price = models.IntegerField(null=False)

    def __str__(self):
        return f"{self.product_name} (기본 수량-{self.quantity}개, 기본 수량의 가격-{self.price}원)"


class OrderStatus(models.Model):
    STATUS_CHOICES = (
        ('order', '주문 완료'),
        ('payment_completed', '입금 완료'),
        ('order_changed', '주문 변경'),
        ('order_canceled', '주문 취소'),
    )

    order = models.OneToOneField('Order', on_delete=models.CASCADE, related_name='status_changes')
    status = models.CharField(max_length=50, choices=STATUS_CHOICES, default='order')
    changed_at = models.DateTimeField(auto_now_add=True)
    
    def __str__(self):
        return f"Order {self.order.id} changed to {self.status} on {self.changed_at}"
    

class Order(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name='orders')
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    order_status = models.CharField(max_length=50, choices=OrderStatus.STATUS_CHOICES, default='order', null=True)
    
    def __str__(self):
        return f"Order {self.id} by {self.user.username}"
    
    def to_dict(self):
        return {
            "id": self.id,
            "user": self.user.username,
            "created_at": self.created_at.isoformat(),
            "updated_at": self.updated_at.isoformat(),
            "order_status": self.order_status,
            "items": [item.to_dict() for item in self.order_items.all()]  # 이 줄을 추가하여 OrderItems를 포함
        }


class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE, related_name='order_items')
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    quantity = models.PositiveIntegerField()
    price = models.DecimalField(max_digits=10, decimal_places=2) 

    def __str__(self):
        return f"{self.quantity} x {self.product.product_name} at {self.price} each"
    
    def to_dict(self):
        return {
            "product_name": self.product.product_name,
            "quantity": self.quantity,
            "price": float(self.price)  # Decimal을 float으로 변환
        }


ModuleNotFoundError: No module named 'orderbot'

In [3]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from django.db import transaction
from django.contrib.auth.models import User
from decimal import Decimal
from products.models import Product, Order, OrderItem

@tool
def fetch_product_list():
    """
    Fetchs a list of products.
    This function retrieves and displays a list of products. 
    """

    product_list = """
    [
        {
            "product_name": "떡케익5호",
            "quantity": 1,
            "price": 54000
        },
        {
            "product_name": "무지개 백설기 케익",
            "quantity": 1,
            "price": 51500
        },
        {
            "product_name": "미니 백설기",
            "quantity": 35,
            "price": 31500
        },
        {
            "product_name": "개별 모듬팩",
            "quantity": 1,
            "price": 13500
        }
    ]
    """
    return product_list


class ToRequestConfirmation(BaseModel):
    """Transfers work to a specialized assistant to request user's approval"""
    product_list: str = Field(description="판매 중인 상품 목록")
    customer_order_request: str = Field(description="고객의 주문 요청 사항")
    message_to_be_confirmed: str = Field(description="고객의 주문 요청 내용에 대해 확인을 요청하는 메시지. 상품명과 상품 가격이 판매 중인 상품 목록과 일치해야 함.")


@tool
def create_order(user_id: int, items :list[dict[str, str | int | float]]):
    """
    Places a new order.
    This function processes a new order request and confirms it by creating an order and its related items in the database.

    Args:
        user_id (int): The ID of the user placing the order.
        items (list[dict[str, str | int | float]]): A list of dictionaries representing the order details. Each dictionary has the following keys:
            - "product_name" (str): The name of the product.
            - "quantity" (int): The quantity of the product.
            - "price" (float): The price of the product.

    Example:
        items = [
            {"product_name": "떡케익5호", "quantity": 2, "price": 54000.0},
            {"product_name": "무지개 백설기 케익", "quantity": 1, "price": 51500.0}
        ]
    """
    print("-"*70)
    print("create_order 진입")
    
    with transaction.atomic():
        # User 객체 가져오기
        user = User.objects.get(id=user_id)
        
        # 주문 생성
        order = Order.objects.create(user=user)
        
        # 주문 상품 생성
        total_price = Decimal('0.00')
        for item in items:
            product = Product.objects.get(product_name=item["product_name"])
            # OrderItem.objects.create(order=order, product=product, quantity=item["quantity"], price=item["price"])
            order.order_items.create(product=product, quantity=item["quantity"], price=item["price"])
            total_price += Decimal(item["price"]) * Decimal(item["quantity"])

        return order
    



ModuleNotFoundError: No module named 'orderbot'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_create_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문을 생성하는 유능한 주문봇이야.
            적절한 도구를 사용해 고객의 요청을 처리해줘..
            고객이 응답이 필요할 때는 도구를 사용하지 말고 고객에게 응답을 부탁해.
            도구 사용 시 도구 사용에 필요한 인자를 모두 정확하게 추출해야 해.
            도구 사용에 필요한 인자는 아래 메시지에서 추출하면 돼.
            
            1. 판매 중인 상품 목록을 제시하지 않았다면 fetch_product_list 도구를 사용해.
            2. 고객이 주문할 품목을 말했다면 TodOrderRequestConfirmation 도구를 사용해.
            3. 고객이 주문을 진행하려는 내용에 동의했다면 create_order 도구를 사용해.

            user_id: {user_info},
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

order_create_related_tools = [fetch_product_list, ToRequestConfirmation]
order_create_tool = [create_order]
order_create_tools = order_create_related_tools + order_create_tool
order_create_runnable = order_create_prompt | llm.bind_tools(order_create_tools)